In [40]:
import time, ast, json, requests
# import xgboost
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import svm, tree


from scipy import stats

sns.set(color_codes=True)

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Load TFT Game Data

In [3]:
game_data = pd.read_csv('./tft_csv/complete_match_data.csv')
match_ids = pd.read_csv('./tft_csv/match_ids.csv')
player_data = pd.read_csv('./tft_csv/player_data.csv')

In [4]:
game_data.drop('Unnamed: 0', axis=1, inplace=True)
match_ids.drop('Unnamed: 0', axis=1, inplace=True)
player_data.drop('Unnamed: 0', axis=1, inplace=True)

# Load Extra TFT Data

In [5]:
tft_items = None
tft_traits = None
tft_champs = None
with open('./tft_extra_data/items.json') as json_file:
    tft_items = json.load(json_file)

with open('./tft_extra_data/traits.json') as json_file:
    tft_traits = json.load(json_file)

with open('./tft_extra_data/champions.json') as json_file:
    tft_champs = json.load(json_file)
    
temp_items = {}
for item in tft_items:
    temp_items[item['id']] = {'name': item['name'],
                              'tier': item['tier']
                             }
tft_items = temp_items

temp_traits = {}
for trait in tft_traits:
    temp_traits[trait['name']] = {'tier': trait['tier']}
    
tft_traits = temp_traits
tft_traits['Set2_Glacial'] = {'tier': 'B'}
tft_traits['Set2_Assassin'] = {'tier': 'A'}
tft_traits['Set2_Blademaster'] = {'tier': 'S'}
tft_traits['Set2_Ranger'] = {'tier': 'A'}
tft_traits['Metal'] = {'tier': 'A'}
tft_traits['Wind'] = {'tier': 'B'}
tft_traits['Soulbound'] = {'tier': 'A'}


# Manipulate Data Columns into Features

In [6]:
# Group Top/Bottom 4 Placements
game_data['game_won'] = [1 if p <= 4 else 0 for p in game_data['placement']]

### Champ + Tier Evaluation Attempt: #1

In [7]:
# Champion Evaluation
unit_powers = []
for u in game_data['units']:
    unit_lst = ast.literal_eval(u)
    unit_power = 0
    for unit in unit_lst:
        name = unit[0]
        rarity = unit[1]
        char_tier = unit[2]
        items = unit[3]
        
        item_power = 0
        for item in items:
            if item not in tft_items:
                continue
            item_data = tft_items[item]
            # print(item_data['name'], item_data['tier'])
            
            if item_data['tier'] == 'S':
                item_power += 3
            elif item_data['tier'] == 'A':
                item_power += 2
            elif item_data['tier'] == 'B':
                item_power += 1
        # print(unit)
        
        unit_power += (1 + rarity) * char_tier + item_power
    unit_powers.append(unit_power)

game_data['team_unit_power'] = unit_powers

In [8]:
# Class Evaluation
class_powers = []
for t in game_data['traits']:
    traits_lst = ast.literal_eval(t)
    class_bonus = 0

    for trait in traits_lst:
        #print(trait)
        trait_name = trait[0]
        trait_active_tier = trait[1]
        trait_power = 0
        
        if tft_traits[trait_name]['tier'] == 'S':
            trait_power += 3
        elif tft_traits[trait_name]['tier'] == 'A':
            trait_power += 2
        elif tft_traits[trait_name]['tier'] == 'B':
            trait_power += 1
        elif tft_traits[trait_name]['tier'] == 'C':
            trait_power += 0.5
        
        if trait_active_tier == 1:
            active_tier_multiplier = 0.4
        else:
            active_tier_multiplier = trait_active_tier/10
        
        class_bonus += trait_power * (1 + active_tier_multiplier)
        
    class_powers.append(class_bonus)
game_data['class_powers'] = class_powers

# Split Data for Test/Train

In [9]:
game_data.head(8)

,puuid,match_id,game_length,gold_left,last_round,level,placement,players_eliminated,time_eliminated,total_damage_to_players,traits,units,game_won,team_unit_power,class_powers
0,HbYsPpRS3X2RaQ8n5Sm9n2gKU80f927PzBk_krw4kdyRG6...,NA1_3249805878,1959.947266,5,31,8,4,0,1714.102783,95,"[('Berserker', 2), ('Desert', 0), ('Electric',...","[('Renekton', 0, 2, []), ('DrMundo', 2, 2, [])...",1,52,16.6
1,OkaQSLZuy-FUjgfc_a770gWAvOtn5WLYcBN7toyVLJsc1d...,NA1_3249805878,1959.947266,0,35,9,2,3,1951.685059,146,"[('Avatar', 1), ('Electric', 3), ('Inferno', 1...","[('Annie', 3, 2, []), ('Zed', 4, 2, [24, 66, 1...",1,91,26.0
2,HGeDGOQrklxGiScizUCLq_BVn5gTYBy-DldOuy38XcaKap...,NA1_3249805878,1959.947266,0,31,7,5,0,1711.923218,72,"[('Berserker', 0), ('Desert', 2), ('Inferno', ...","[('Nocturne', 2, 2, [19, 19, 23]), ('Sivir', 2...",0,52,14.8
3,VyCpwiJ3MifHEREwjj0cSAXHQIjFGd7UtWMBG9ZvCdlGso...,NA1_3249805878,1959.947266,30,26,7,7,0,1445.723633,55,"[('Inferno', 1), ('Light', 0), ('Mountain', 0)...","[('Kindred', 2, 2, [44, 22, 39]), ('Malzahar',...",0,57,20.8
4,UsiShYgxH3X5_pNeJjE-EpBhbR4vevorLcs83aI8ZSQFaZ...,NA1_3249805878,1959.947266,4,33,8,3,0,1819.271362,129,"[('Avatar', 1), ('Desert', 1), ('Inferno', 0),...","[('Yasuo', 1, 2, []), ('Sivir', 2, 2, [25, 46,...",1,73,23.0
5,BXPswgAxjGfAfQHvaF-m7CtUdbBvZQhcviLqZwZDyDZbDn...,NA1_3249805878,1959.947266,50,28,8,6,0,1567.998779,80,"[('Inferno', 1), ('Light', 0), ('Mage', 0), ('...","[('Nami', 4, 1, [25, 57]), ('Thresh', 1, 2, [6...",0,61,22.4
6,zqelCdkKSSNGoCJA9msW7sP-wmzIW0HPG8b8ElUMwLCIMl...,NA1_3249805878,1959.947266,8,24,7,8,0,1324.369019,31,"[('Desert', 1), ('Inferno', 0), ('Light', 0), ...","[('Aatrox', 2, 2, []), ('Nocturne', 2, 2, [19,...",0,48,13.6
7,BXZMHIfauv8M3BuaqO6wZnxQXrN2flWKj1GhJ6nNQeP3DD...,NA1_3249805878,1959.947266,1,35,9,1,4,1951.685059,176,"[('Avatar', 1), ('Crystal', 0), ('Electric', 0...","[('Malphite', 3, 2, []), ('Lux', 5, 1, []), ('...",1,89,31.4


In [28]:
# feature_columns = ['gold_left', 'level','players_eliminated', 'total_damage_to_players']
feature_columns = ['team_unit_power',
                   'class_powers',
                   'players_eliminated',
                   'total_damage_to_players',
#                    'gold_left',
#                    'level'
                  ]
X = game_data[feature_columns]
y = game_data['game_won']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

### Logistic Regression

In [29]:
# Feature Elimination
logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(X_train, y_train)
print(rfe.support_)
print(rfe.ranking_)

[ True  True  True  True]
[1 1 1 1]


In [30]:
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.325575
         Iterations 8
                             Results: Logit
Model:                 Logit              Pseudo R-squared:   0.530     
Dependent Variable:    game_won           AIC:                10509.7574
Date:                  2020-01-03 17:01   BIC:                10540.5106
No. Observations:      16128              Log-Likelihood:     -5250.9   
Df Model:              3                  LL-Null:            -11179.   
Df Residuals:          16124              LLR p-value:        0.0000    
Converged:             1.0000             Scale:              1.0000    
No. Iterations:        8.0000                                           
------------------------------------------------------------------------
                         Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
------------------------------------------------------------------------
team_unit_power         -0.0838   0.0031 -27.1395 0

In [31]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))


Accuracy of logistic regression classifier on test set: 0.89


In [32]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89      2408
           1       0.90      0.88      0.89      2431

    accuracy                           0.89      4839
   macro avg       0.89      0.89      0.89      4839
weighted avg       0.89      0.89      0.89      4839



### Decision Tree

In [33]:
model_dt = tree.DecisionTreeClassifier()
model_dt.fit(X_train, y_train)
y_pred= model_dt.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy of %s is %s"%(model_dt, acc))


Accuracy of DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best') is 0.9253978094647655


In [36]:
pd.Series(model_dt.feature_importances_,index=feature_columns).sort_values(ascending=False)

total_damage_to_players    0.760207
class_powers               0.134313
team_unit_power            0.088713
players_eliminated         0.016767
dtype: float64

### Random Forest

In [37]:
model_rfc = RandomForestClassifier(n_estimators=200)
model_rfc.fit(X_train, y_train)
y_pred= model_rfc.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy of %s is %s"%(model_rfc, acc))

Accuracy of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False) is 0.9377970655094028


In [38]:
pd.Series(model_rfc.feature_importances_,index=feature_columns).sort_values(ascending=False)

total_damage_to_players    0.529447
team_unit_power            0.228302
class_powers               0.129328
players_eliminated         0.112922
dtype: float64

### SVM

In [39]:
# SVM
model_svm = svm.SVC(kernel='linear') # linear, rbf, sigmoid, polynomial
model_svm.fit(X_train, y_train)
y_pred= model_svm.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy of %s is %s"%(model_svm, acc))


Accuracy of SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False) is 0.8902665840049597


In [17]:
model_svm.coef_, feature_columns

(array([[ 0.04701779, -0.01657947,  0.51616203,  0.05305165]]),
 ['team_unit_power',
  'class_powers',
  'players_eliminated',
  'total_damage_to_players'])

In [ ]:
sns.barplot(x='players_eliminated', y='game_won', data=game_data)

In [ ]:
sns.barplot(x='total_damage_to_players', y='game_won', data=game_data)

In [ ]:
sns.distplot(game_data['level'], hist=True, kde=True, label = 'Level')

In [ ]:
sns.distplot(game_data['players_eliminated'], hist=False, kde=True, label = 'players eliminated')

In [ ]:
sns.distplot(game_data['team_unit_power'], hist=False, kde=True, label = 'players eliminated')

In [ ]:
sns.distplot(game_data['class_powers'], hist=False, kde=True, label = 'players eliminated')

In [ ]:
plt.scatter(x=game_data['team_unit_power'], y=game_data['class_powers'])
plt.xlabel("Team Unit Power")
plt.ylabel("Class Power")
plt.title("Class x Team Unit")